In [1]:
import requests
from lxml import html
import os
import sys
import logging # for logging
import shutil #to delete the directory contents
import zipfile
from zipfile import ZipFile
from io import BytesIO
import pandas as pd
import zipfile
import numpy as np
import json
import logging
import logging.handlers
import time
import glob
import csv
import seaborn as sns
from numbers import Number

In [3]:
import logging
import logging.handlers

logger=logging.getLogger(__name__)
logger.setLevel(logging.INFO)

logfile1 = time.strftime("%Y-%m-%d_%H_%M_%S"+".log")
print (logfile1)
handler= logging.FileHandler(logfile1)
handler.setLevel(logging.INFO)

formatter= logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

2017-07-21_23_04_21.log


In [6]:
logger.info('Reading Json')

with open('mode_config.json') as data:
    payload=json.load(data)

In [ ]:
logger.info('Logging in and Downloading Zip ')
filename= 'historical_zips'
path = '/historical_zips'
path1= os.getcwd()+path
if not os.path.exists(path1):
    os.makedirs(path1)
with requests.session() as s:
    login_url = "https://freddiemac.embs.com/FLoan/secure/auth.php"
    result = s.post(
        login_url, 
        data = payload, 
        headers = dict(referer=login_url)
    )
    result.content
    url = 'https://freddiemac.embs.com/FLoan/Data/download.php'
    agreement_payload={
        "accept":"Yes",
        "action":"acceptTandC",
        "acceptSubmit":"Continue"
        }
    result1 = s.post(
        url, 
        agreement_payload,
        headers = dict(referer = url)
    )
    tree = html.fromstring(result1.content)
    all_links= tree.findall('.//a')
    for link in all_links[0:75]:
        href=link.get("href")
        if "hist" in href:
            url= 'https://freddiemac.embs.com/FLoan/Data/'+href
            r = s.get(url,stream=True)
#             print(r)
            with open(os.path.join(path1,link.text), 'wb') as f:
#                 print(link.text)
                for chunk in r.iter_content(chunk_size=1024):
                    if chunk: # filter out keep-alive new chunks
                        f.write(chunk)

                    
logger.info('Successfully downloaded zip files ')

In [9]:
logger.info('Checking if a folder exists and creating a folder for saving unzipped files')
unzip_folder='hist_unzip'
path_unzip = '/hist_unzip'
path_to_unzip= os.getcwd()+path_unzip
if not os.path.exists(path_to_unzip):
    os.makedirs(path_to_unzip)